In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [20]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup

class Scraper:
    def __init__(self, keywords, pages):
        self.keywords = keywords
        self.pages = pages

    def fetch(self, base_url):
        self.base_url = base_url

        self.params = {
            'query': self.keywords,
            'sortby': 'time',
            'page': 2
        }

        self.headers = {
            'sec-ch-ua': '"(Not(A:Brand";v="8", "Chromium";v="99", "Google Chrome";v="99"',
            'sec-ch-ua-platform': "Linux",
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.35 Safari/537.36'
        }

        self.response = requests.get(self.base_url, params=self.params, headers=self.headers)

        return self.response

    def get_articles(self, response):
        article_lists = []

        for page in range(1, int(self.pages) + 1):
            url = f"{self.base_url}?query={self.keywords}&sortby=time&page={page}"

            page = requests.get(url)
            soup = BeautifulSoup(page.text, "html.parser")

            articles = soup.find_all("article")

            for article in articles:
                category = article.find("span", {"class": "category"}).get_text()

                if 'detikFinance' in category:
                    title = article.find("h2", {"class": "title"}).get_text()
                    published_time = article.find("span", {"class": "date"}).get_text().split(",")[1]
                    href = article.find("a")["href"]

                    # Ambil konten dari halaman berita penuh
                    full_article_page = requests.get(href)
                    full_article_soup = BeautifulSoup(full_article_page.text, "html.parser")

                    # Check if the div with class "detail__body-text" exists
                    full_article_div = full_article_soup.find("div", {"class": "detail__body-text"})
                    content = full_article_div.get_text() if full_article_div else None

                    article_lists.append({
                        "subject": "ekonomi",  # Ganti dengan kategori yang sesuai
                        "url": href,
                        "title": title,
                        "post_date": published_time,
                        "post_by": "DetikFinance",  # Ganti dengan sumber berita yang sesuai
                        "content": content
                    })

        self.articles = article_lists

        try:
            self.show_results()
        except Exception as e:
            print(e)
        finally:
            print()
            print("[~] Scraping finished!")
            print(f"[~] Total Articles: {len(self.articles)}")

        return self.articles


    def save_to(self, file_format="csv"):
        time_scrape = datetime.now().strftime("%m%d%Y_%H%M%S")

        df = pd.DataFrame(self.articles)

        file_name = f"result_{self.keywords}_{time_scrape}"
        if file_format == "json":
            file_name += ".json"
            df.to_json(file_name, orient="records")
            print(f"[~] Result saved to '{file_name}'")
        elif file_format in ["excel", "xlsx"]:
            file_name += ".xlsx"
            df.to_excel(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")
        elif file_format == "csv":
            file_name += ".csv"
            df.to_csv(file_name, index=False)
            print(f"[~] Result saved to '{file_name}'")
        else:
            print("[~] Unsupported file format. Please use 'json', 'excel', or 'csv'.")

if __name__ == '__main__':
    keywords = input("[~] Keywords     : ")
    pages = input("[~] Total Pages  : ")
    base_url = f"https://www.detik.com/search/searchall"

    scrape = Scraper(keywords, pages)
    response = scrape.fetch(base_url)
    articles = scrape.get_articles(response)

    try:
        ask = input("[~] Do you want to save the results? [y/n]: ").lower()
        if ask == 'y':
            file_format = input("[~] Save to file format? [json/excel/csv]     : ").lower()
            scrape.save_to(file_format=file_format)
        elif ask == 'n':
            scrape.show_results()
    except Exception as e:
        print(e)
    finally:
        print("[~] Program Finished")


[~] Keywords     : ekonomi
[~] Total Pages  : 1000
'Scraper' object has no attribute 'show_results'

[~] Scraping finished!
[~] Total Articles: 2495
[~] Do you want to save the results? [y/n]: y
[~] Save to file format? [json/excel/csv]     : csv
[~] Result saved to 'result_ekonomi_12112023_182835.csv'
[~] Program Finished


In [21]:
data = pd.read_csv('result_ekonomi_12112023_182835.csv')
data

,subject,url,title,post_date,post_by,content
0,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,20 KEK RI Bakal Gelar Rakernas Bahas Percepata...,11 Des 2023 16:41 WIB,DetikFinance,\n\n\nJakarta - Sebanyak 20 Kawasan Ekonomi Kh...
1,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,"Bicara Pemerataan, Ganjar Singgung Akses Tol h...",11 Des 2023 16:32 WIB,DetikFinance,\n\n\nJakarta - Calon Presiden (capres) Ganjar...
2,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,"Jokowi Kumpulkan Para Menteri di Istana, Bahas...",11 Des 2023 15:48 WIB,DetikFinance,\n\n\nJakarta - Presiden Joko Widodo (Jokowi) ...
3,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,Besok Harbolnas! Sudah Bisa Belanja Lagi di Ti...,11 Des 2023 15:20 WIB,DetikFinance,\n\n\nJakarta - TikTok Shop hadir lagi. Hal in...
4,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,"Dapat Nilai Bagus dari Kemenkumham, Menteri KP...",11 Des 2023 15:05 WIB,DetikFinance,\n\n\nJakarta - Kementerian Kelautan dan Perik...
...,...,...,...,...,...,...
2490,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,Luhut Temui Bos IMF Usai Hilirisasi RI Disenti...,11 Agu 2023 12:56 WIB,DetikFinance,\n\n\nJakarta - Menteri Koordinator Bidang Kem...
2491,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,Luhut Ungkap Startup RI Terbanyak Nomor 6 di D...,11 Agu 2023 12:31 WIB,DetikFinance,\n\n\nJakarta - Menteri Koordinator Bidang Kem...
2492,ekonomi,https://finance.detik.com/moneter/d-6870675/by...,Bye-bye Dolar AS! RI Mulai Pakai Mata Uang Lai...,11 Agu 2023 12:26 WIB,DetikFinance,\n\n\nJakarta - Indonesia terus mengembangkan ...
2493,ekonomi,https://finance.detik.com/berita-ekonomi-bisni...,"Peringatan Hari UMKM Nasional di Solo Dibuka, ...",11 Agu 2023 11:20 WIB,DetikFinance,\nJakarta - Perayaan Hari UMKM Nasional Expo ...
